In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pytz
import pandas as pd
import requests
from os import listdir
from os.path import isfile, isdir
from datetime import datetime
from urllib.parse import unquote
from tqdm.autonotebook import tqdm
from module.elasticsearch_utils import ElasticSearchUtils

/home/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [2]:
host_info = {
    'host': 'https://corpus.ncsoft.com:9200',
    'http_auth': 'crawler:crawler2019',
    'index': 'crawler-youtube-live-chat',
    'split_index': True,
}

utils = ElasticSearchUtils(**host_info)

/home/.local/lib/python3.6/site-packages/elasticsearch/connection/http_urllib3.py:147: UserWarning: When using `ssl_context`, all other SSL related kwargs are ignored
  "When using `ssl_context`, all other SSL related kwargs are ignored"


In [44]:
def simplify(doc, title):
    """ """
    try:
        chat_list = doc['response']['continuationContents']['liveChatContinuation']['actions']
    except Exception as e:
        return []
    
    tz = pytz.timezone('Asia/Seoul')

    result = []
    for act_item in chat_list:
        if 'replayChatItemAction' not in act_item:
            return []

        for act in act_item['replayChatItemAction']['actions']:
            try:
                chat_text = act['addChatItemAction']['item']['liveChatTextMessageRenderer']
            except Exception as e:
                continue
                
            item = {
                '_id': unquote(chat_text['id']),
                'title': title,
                'video_offset': int(act_item['replayChatItemAction']['videoOffsetTimeMsec']),
                'time_stamp': chat_text['timestampText']['simpleText'],
                'author': chat_text['authorName']['simpleText'],
                'text': '\n'.join([v['text'].strip() for v in chat_text['message']['runs'] if 'text' in v]),
            }

            try:
                timestemp = int(chat_text['timestampUsec'][0:-3]) / 1000.0

                dt = datetime.fromtimestamp(timestemp)
                item['date'] = dt.astimezone(tz).isoformat()
            except Exception as e:
                print('error at parse date', [timestemp, e])

            result.append(item)

    return result


def extract_chat_text(data_path, title):
    """ """
    doc_list = []
    for f_name in listdir('{}/{}'.format(data_path, title)):
        f = '{}/{}/{}'.format(data_path, title, f_name)
        
        if isdir(f):
            continue

        with open(f, 'r') as fp:
            doc = json.load(fp)

        doc_list += simplify(doc=doc, title=title)

    return doc_list

In [45]:
data_path = 'data/youtube-live-chat'
data_path = 'data/youtube-live-chat/[똘끼 풀영상]모음'

pbar = tqdm(listdir(data_path))
for d in pbar:
    if isfile('{}/{}'.format(data_path, d)):
        continue
        
    pbar.set_description(d)
    
    doc_list = extract_chat_text(data_path=data_path, title=d)
    if len(doc_list) == 0:
        continue
        
    for doc in doc_list:
        utils.save_document(document=doc)
    utils.flush()
    
    df = pd.DataFrame(doc_list)

    df.sort_values('video_offset', inplace=True)
    df.to_excel('{}/{}.xlsx'.format(data_path, d), index=False)